# Попытка улучшить показатели лучшей реализации с ошибкой в ~12.93% RMSE
Прошлый подход с попытками найти гиперпараметры через optuna не дал особых результатов и установленную точность так и не побили. Попробуем провести фича-инжиниринг, а именно: поиграть с тем как мы заполняем пропуски (внедрив заполнение не по mean/median, а через Predictive Imputer или KNN) и т.п.

In [1]:
from IPython.lib.deepreload import reload
from xgboost import train
%load_ext autoreload
%autoreload 2

import joblib
import numpy as np
import optuna
import pandas as pd

from catboost import CatBoostRegressor
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, root_mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, PowerTransformer, RobustScaler, QuantileTransformer, PolynomialFeatures, KBinsDiscretizer

# Вместо одного фиксированного разбиения на train/test используем стабильную стратегию кросс-валидации.
# Используем тут Cross-validation, потому что:
# 	•	нужно надёжно сравнить несколько разных моделей или гиперпараметров и понять, какая модель стабильнее и лучше в целом.
# 	•	хотим избежать случайных удач или провалов, связанных с конкретным разбиением на train/test.
# 	•	выбираем модель или гиперпараметры, которые потом будешь использовать для финального сабмишна на Kaggle.
# Делаем эту оценку, чтобы в дальнейших блокнотах-улучшениях сравнивать более корректно.
from sklearn.model_selection import KFold, cross_val_score, train_test_split

# Используем IterativeImputer:
# 	•	Он итеративно заполняет все пропуски сразу.
# 	•	Работает одновременно со всеми признаками, учитывая связи между ними.
# 	•	Не требует ручного управления порядком заполнения.
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import HistGradientBoostingRegressor

from utils.data_manager import DataManager
from utils.model_manager import ModelManager

## 1. Загрузка данных

In [2]:
data_path = 'data/home-data-for-ml-course'
train_data = pd.read_csv(data_path + '/train.csv')
test_data = pd.read_csv(data_path + '/test.csv')

## 2. Предобработка данных 

In [3]:
dm = DataManager()

### 2.1. Удаление ненужных столбцов

In [4]:
intuitively_bad_features = [
    # 'LotShape',  # Общая форма участка
    # 'LandContour',  # Рельеф участка
    # 'LotConfig',  # Конфигурация участка
    # 'LandSlope',  # Уклон участка
    # 'MiscFeature',
    # 'MiscVal',
]
bad_columns = dm.get_all_nan_cols(train_data)
bad_columns.append('Id')
bad_columns.extend(intuitively_bad_features)
bad_columns

['Id']

In [5]:
# best_features_by_catboost = ['OverallQual', 'GrLivArea', 'GarageCars', 'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', 'LotArea', 'BsmtQual', 'Fireplaces', '2ndFlrSF', 'YearBuilt', 'GarageYrBlt', 'ExterQual', 'OpenPorchSF', 'Neighborhood', 'KitchenQual', 'BsmtExposure', 'OverallCond', 'TotRmsAbvGrd', 'YearRemodAdd', 'GarageArea', 'GarageFinish', 'BsmtFinType1', 'LotFrontage', 'HalfBath', 'FullBath', 'SaleCondition', 'SaleType', 'Condition1', 'BedroomAbvGr', 'HeatingQC', 'MoSold', 'BsmtFullBath', 'BsmtUnfSF', 'CentralAir', 'MSSubClass', 'Exterior1st', 'ScreenPorch', 'WoodDeckSF', 'MSZoning', 'MasVnrArea', 'GarageType', 'HouseStyle', 'RoofStyle', 'YrSold', 'MasVnrType', 'Functional', 'Exterior2nd', 'PoolArea', 'BldgType', 'FireplaceQu', 'ExterCond', 'KitchenAbvGr', 'EnclosedPorch', 'PavedDrive', 'Electrical', 'Alley', 'Condition2', 'BsmtFinSF2', 'BsmtCond', 'BsmtFinType2', '3SsnPorch', 'Foundation', 'RoofMatl', 'GarageQual', 'Fence', 'PoolQC', 'Heating', 'BsmtHalfBath', 'LowQualFinSF', 'GarageCond', 'Street', 'Utilities']
# best_features_by_catboost = ['OverallQual', 'GrLivArea', 'GarageCars', 'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', 'LotArea', 'BsmtQual', 'Fireplaces', '2ndFlrSF', 'YearBuilt', 'GarageYrBlt', 'ExterQual', 'OpenPorchSF', 'Neighborhood', 'KitchenQual', 'BsmtExposure', 'OverallCond', 'TotRmsAbvGrd', 'YearRemodAdd', 'GarageArea', 'GarageFinish', 'BsmtFinType1', 'LotFrontage', 'HalfBath', 'FullBath', 'SaleCondition', 'SaleType', 'Condition1', 'BedroomAbvGr', 'HeatingQC', 'MoSold', 'BsmtFullBath', 'BsmtUnfSF', 'CentralAir', 'MSSubClass', 'Exterior1st', 'ScreenPorch', 'WoodDeckSF', 'MSZoning', 'MasVnrArea', 'GarageType', 'HouseStyle', 'RoofStyle', 'YrSold', 'MasVnrType', 'Functional', 'Exterior2nd', 'PoolArea', 'BldgType', 'FireplaceQu', 'ExterCond', 'KitchenAbvGr', 'EnclosedPorch', 'PavedDrive', 'Electrical', 'Alley', 'Condition2', 'BsmtFinSF2', 'BsmtCond', 'BsmtFinType2', ]
# best_features_by_catboost.append('SalePrice')
# Подход с получением топ-фич из катбуста после обучения модели и потом с ручными экспериментами по заданию топ 20-40-50 - не дает результатов. Показания ухудшаются как правило, ручной перебор фич - плохо как мне кажется
best_features_by_catboost = None

In [6]:
if best_features_by_catboost:
    train_data = train_data.drop(columns=bad_columns)[best_features_by_catboost]
else:
    train_data = train_data.drop(columns=bad_columns)
train_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [7]:
X, y = dm.split_data_set_to_x_y(train_data, 'SalePrice')
print(X.shape, y.shape)

(1460, 79) (1460,)


### 2.2. Начнем изучать данные, для начала посмотрим, что там с пропусками и количество оных

In [27]:
# Анализ пропущенных значений
missing_info = dm.analyze_missing_values(X)
print("\nАнализ пропущенных значений по колонкам:")
missing_info


Анализ пропущенных значений по колонкам:


,Количество пропусков,Процент пропусков (%)
PoolQC,1453,99.52
Alley,1369,93.77
Fence,1179,80.75
MasVnrType,872,59.73
FireplaceQu,690,47.26
...,...,...
HeatingQC,0,0.00
MSZoning,0,0.00
1stFlrSF,0,0.00
2ndFlrSF,0,0.00


Мы видим, что у нас есть два признака: PoolQC, Alley, Fence которые почти отсутствуют во всем датасете. А несут они в себе такую нагрузку: PoolQC - качество бассейна, Alley - тип подъезда к участку; Fence - тип забора. Если не заполнены поля - значит этого просто у дома нет. Поэтому попробуем заменить их на бинарный признак: Has_Pool_or_Alley_or_Fence

In [28]:
# # Создаем новый столбец Has_Pool_or_Alley_or_Fence
# X['Has_Pool_or_Alley_or_Fence'] = 'No'
# 
# # Устанавливаем значение 1, если хотя бы один из столбцов не пустой
# # Для обработки как NaN, так и пустых строк
# mask = ((X[['PoolQC', 'Alley', 'Fence']].notna()) & 
#         (X[['PoolQC', 'Alley', 'Fence']] != '')).any(axis=1)
# X.loc[mask, 'Has_Pool_or_Alley_or_Fence'] = 'Yes'
# 
# X = X.drop(columns=['PoolQC', 'Alley', 'Fence'])

# По результату обучения модели видим после такой операции увеличение ошибки. пробуем просто удалить эти столбцы
# пробовал делать и 0/1 значения, а сейчас оставил 'yes'/'no', чтобы они категорийными посчитались

In [29]:
# X = X.drop(columns=['PoolQC', 'Alley', 'Fence'])
# # Если просто удалить - то еще больше увеличится ошибка. Тоже не вариант.

In [30]:
# Попробуем заменить каждое из трех полей выше на бинарные
# X['Has_Pool'] = 0
# X['Has_Alley'] = 0
# X['Has_Fence'] = 0
# 
# pool_mask = ((X[['PoolQC']].notna()) & (X[['PoolQC']] != '')).any(axis=1)
# alley_mask = ((X[['Alley']].notna()) & (X[['Alley']] != '')).any(axis=1)
# fence_mask = ((X[['Fence']].notna()) & (X[['Fence']] != '')).any(axis=1)
# 
# X.loc[pool_mask, 'Has_Pool'] = 1
# X.loc[alley_mask, 'Has_Alley'] = 1
# X.loc[fence_mask, 'Has_Fence'] = 1
# 
# X = X.drop(columns=['PoolQC', 'Alley', 'Fence'])

# Результат стал лучше, чем после полных удалений или объединений 3 в 1, но не улучшил исходные показатели, отказываемся от идей с бинарными признаками

In [31]:
# X[X['Has_Pool'] == 1]

In [32]:
# Попробуем заполнить пропуски в PoolQC, Alley, Fence значением "No" (т.е. "нет")
# X['PoolQC'] = X['PoolQC'].fillna('No')
# X['Alley'] = X['Alley'].fillna('No')
# X['Fence'] = X['Fence'].fillna('No')

# Данный метод тоже никак не улучшил результаты

In [33]:
# Анализ пропущенных значений только числовых
numeric_columns = X.select_dtypes(include=['float64', 'int64']).columns
missing_info = dm.analyze_missing_values(X[numeric_columns])
print("\nАнализ пропущенных значений по колонкам:")
missing_info


Анализ пропущенных значений по колонкам:


,Количество пропусков,Процент пропусков (%)
LotFrontage,259,17.74
GarageYrBlt,81,5.55
MasVnrArea,8,0.55
GarageArea,0,0.00
BedroomAbvGr,0,0.00
KitchenAbvGr,0,0.00
TotRmsAbvGrd,0,0.00
Fireplaces,0,0.00
GarageCars,0,0.00
MSSubClass,0,0.00


In [8]:
missing_numeric_features = [
    'LotFrontage',
    'GarageYrBlt',
    'MasVnrArea',
]

In [35]:
# categorical_features = X.select_dtypes(include=['object']).columns
# numeric_features = X.select_dtypes(include=['float64', 'int64']).columns
# 
# # Предварительно кодируем категориальные признаки, так как IterativeImputer принимает только числа
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', 'passthrough', numeric_features),
#         ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
#     ])
# 
# # Создаем пайплайн с препроцессором и имьютером
# pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('imputer', IterativeImputer(
#         estimator=RandomForestRegressor(n_estimators=50, random_state=42),
#         max_iter=10,
#         random_state=42
#     ))
# ])

In [36]:
# Если хочется посмотреть на то, что вышло
# X_imputed_array = pipeline.fit_transform(X)
# 
# # Восстанавливаем DataFrame после заполнения
# feature_names = numeric_features.tolist() + list(pipeline.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(categorical_features))
# X_imputed = pd.DataFrame(X_imputed_array, columns=feature_names, index=X.index)
# 
# X_imputed.head()

### 2.2. Нормализация данных через ColumnTransformer и Pipeline
В данном кейсе мы реализуем Заполнение числовых пропусков с помощью модели (Predictive imputation). Т.е. то, что пропущено в числовых признаках - будем заполнять не медианой или средним, а будем обучать модель, которая будет предсказывать пропуски (IterativeImputer + RandomForestRegressor)

In [9]:
RANDOM_STATE = 42

In [10]:
# Получение числовых колонок
numeric_columns = X.select_dtypes(include=['float64', 'int64']).columns

# Получение нечисловых колонок (всех остальных)
non_numeric_columns = X.select_dtypes(exclude=['float64', 'int64']).columns

numeric_columns, non_numeric_columns

(Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
        'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
        'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
        'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
        'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
        'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
        'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
        'MoSold', 'YrSold'],
       dtype='object'),
 Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
        'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
        'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
        'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
        'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
        'Heating', 'Hea

In [11]:
# Создаем preprocessor с разными трансформерами для разных типов данных
# Числовые данные пропущенные предсказываем с помощью модели RandomForestRegressor

# Пайплайн для числовых признаков (итеративное заполнение)

numeric_transformer = Pipeline(steps=[
    ('imputer', IterativeImputer(
        estimator=RandomForestRegressor(n_estimators=50, random_state=RANDOM_STATE),
        max_iter=10,
        random_state=RANDOM_STATE
    ))
])
# С RandomForestRegressor - все ок, получили некий прирост, попробуем заменить на HistGradientBoostingRegressor
# С ним результат вышел хуже
# numeric_transformer = Pipeline(steps=[
#     ('imputer', IterativeImputer(
#         estimator=HistGradientBoostingRegressor(max_iter=200, random_state=42),
#         max_iter=10,
#         random_state=42
#     ))
# ])

# Пайплайн для категориальных признаков (заполнение частым значением и кодирование)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Объединяем препроцессоры
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, non_numeric_columns)
    ],
    remainder='drop'
)

default_params = {
    'iterations': 1000, 
    'learning_rate': 0.05, 
    'depth': 6, 
    'loss_function': 'RMSE', 
    'verbose': 0, 
    'random_seed': RANDOM_STATE,
}

# Итоговый пайплайн с моделью
# feature_selector - SequentialFeatureSelector тюнит фичи, удаляя по одной до тех пор, пока на очередном удалении не получит метрику хуже или такую же как была. Если указано direction='backward', то стартует со всех фич и удаляет по одной. Если указать 'forward', то стартанет с пустого набора и будет добавлять по одному. С backward больше шансов найти лучшую комбинацию.
# (!!!!!!!!) ОЧЕНЬ ДОЛГИЙ ПРОЦЕСС С SFS - БОЛЕЕ 17 ЧАСОВ (!) ОСТОРОЖНО (!!!!!!!!)
final_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('feature_selector', SequentialFeatureSelector(
        estimator=CatBoostRegressor(**default_params),
        n_features_to_select='auto',  # 'auto' - оптимально подбирает само
        direction='backward',         # Удаление признаков по одному назад
        scoring='neg_root_mean_squared_error',  # метрика для регрессии RMSE
        cv=5,                         # количество фолдов на кросс-валидации
        n_jobs=-1                     # параллельная работа на всех ядрах
    )),
    ('model', CatBoostRegressor(**default_params))
])

### 2.3. Разбиение данных на обучающую и тестовую выборки

In [12]:
# Катбуст сам выбирает даныне для валидации, поэтому отсекаем данные только для теста
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_STATE)

# Проверка размера полученных наборов
print(f'Train size: {len(X_train)}')
print(f'Test size: {len(X_test)}')

Train size: 1022
Test size: 438


In [41]:
# Обучение модели
final_pipeline.fit(X_train, y_train)

# Предсказание и оценка
y_pred_cat = final_pipeline.predict(X_test)
mse_cat = mean_squared_error(y_test, y_pred_cat)
rmse_cat = root_mean_squared_error(y_test, y_pred_cat)
r2_cat = r2_score(y_test, y_pred_cat)

print("\n=== CatBoost ===")
print(f'RMSE: {rmse_cat:.4f}')
print(f'R² (коэффициент детерминации): {r2_cat:.4f}')

# Используем кросс-валидацию (cross-validation), чтобы результаты были надежными и не зависели от случайного разбиения.
# scores = cross_val_score(final_pipeline, X, y, scoring='neg_root_mean_squared_error', cv=5)
# rmse_scores = -scores
# print("Средний RMSE на кросс-валидации:", rmse_scores.mean())


=== CatBoost ===
RMSE: 24191.0591
R² (коэффициент детерминации): 0.9161


#### К чему стремимся:
=== CatBoost ===
- RMSE: 23465.1123
- R² (коэффициент детерминации): 0.9211
- Средний RMSE на кросс-валидации: 25687.30318717033

In [43]:
# Получаем список фич, отобранных SequentialFeatureSelector-ом
# Добираемся до самого SFS
sfs = final_pipeline.named_steps['feature_selector']

# Получаем маску выбранных фич
support_mask = sfs.get_support()

# Получаем имена фич после препроцессора
# Важно: нужно применить препроцессор отдельно
X_processed = preprocessor.fit_transform(X_train)
if hasattr(preprocessor, 'get_feature_names_out'):
    feature_names = preprocessor.get_feature_names_out()
else:
    feature_names = [f"feature_{i}" for i in range(X_processed.shape[1])]

# Выбранные признаки:
selected_features = feature_names[support_mask]

In [52]:
len(selected_features), len(feature_names)

(131, 261)

In [49]:
pd.DataFrame(selected_features).to_csv('selected_features.csv')

## Пайплайн делаем по уму, чтобы не ждать кучу часов SFS, разносим все по шагам и переиспользуем

In [13]:
# Обучаем препроцессор отдельно
preprocessor.fit(X_train)

# Преобразуем данные
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Получаем имена признаков после препроцессинга
feature_names = preprocessor.get_feature_names_out()

# Обучаем SequentialFeatureSelector один раз
sfs = SequentialFeatureSelector(
    estimator=CatBoostRegressor(**default_params),
    n_features_to_select='auto',
    direction='backward',
    scoring='neg_root_mean_squared_error',
    cv=5,
    n_jobs=-1
)
sfs.fit(X_train_processed, y_train)

# Сохраняем препроцессор и отбор фич
joblib.dump(preprocessor, 'preprocessor.pkl')
joblib.dump(sfs, 'feature_selector.pkl')

['feature_selector.pkl']

In [18]:
# Проверяем, какие фичи были отобраны
# Загружаем сохранённые шаги
preprocessor = joblib.load('preprocessor.pkl')
sfs = joblib.load('feature_selector.pkl')
# Получаем маску выбранных фич
support_mask = sfs.get_support()

# Получаем имена фич после препроцессора
if hasattr(preprocessor, 'get_feature_names_out'):
    feature_names = preprocessor.get_feature_names_out()
else:
    feature_names = [f"feature_{i}" for i in range(X_processed.shape[1])]

# Выбранные признаки:
selected_features = feature_names[support_mask]
selected_features, len(selected_features), len(feature_names)

(array(['num__OverallQual', 'num__OverallCond', 'num__YearBuilt',
        'num__YearRemodAdd', 'num__1stFlrSF', 'num__LowQualFinSF',
        'num__GrLivArea', 'num__BsmtFullBath', 'num__FullBath',
        'num__HalfBath', 'num__KitchenAbvGr', 'num__Fireplaces',
        'num__GarageCars', 'num__EnclosedPorch', 'num__3SsnPorch',
        'num__ScreenPorch', 'num__PoolArea', 'cat__MSZoning_C (all)',
        'cat__MSZoning_FV', 'cat__Street_Pave', 'cat__Alley_Grvl',
        'cat__LotShape_IR2', 'cat__LotShape_IR3', 'cat__LandContour_Bnk',
        'cat__LandContour_HLS', 'cat__LandContour_Low',
        'cat__LandContour_Lvl', 'cat__LotConfig_FR2', 'cat__LandSlope_Sev',
        'cat__Neighborhood_BrDale', 'cat__Neighborhood_BrkSide',
        'cat__Neighborhood_Crawfor', 'cat__Neighborhood_Edwards',
        'cat__Neighborhood_Gilbert', 'cat__Neighborhood_IDOTRR',
        'cat__Neighborhood_MeadowV', 'cat__Neighborhood_Mitchel',
        'cat__Neighborhood_NPkVill', 'cat__Neighborhood_NWAmes',
 

In [19]:
# Загружаем и собираем финальный пайплайн для быстрого обучения
# Кастомный трансформер для применения сохранённого препроцессора и SFS
class PreprocessAndSelect(BaseEstimator, TransformerMixin):
    def __init__(self, preprocessor, selector):
        self.preprocessor = preprocessor
        self.selector = selector
        
    def fit(self, X, y=None):
        return self  # уже обучены
        
    def transform(self, X):
        X_transformed = self.preprocessor.transform(X)
        X_selected = self.selector.transform(X_transformed)
        return X_selected

# Загружаем сохранённые шаги
preprocessor = joblib.load('preprocessor.pkl')
sfs = joblib.load('feature_selector.pkl')

# Собираем финальный пайплайн
final_fast_pipeline = Pipeline([
    ('preprocess_and_select', PreprocessAndSelect(preprocessor, sfs)),
    ('model', CatBoostRegressor(**default_params))
])

# Обучение финальной модели (быстро!)
final_fast_pipeline.fit(X_train, y_train)

# Предсказание
y_pred = final_fast_pipeline.predict(X_test)

# Метрики
print("RMSE:", root_mean_squared_error(y_test, y_pred))
print("R²:", r2_score(y_test, y_pred))

RMSE: 24789.95229804395
R²: 0.9119327203112025


In [20]:
# Сохраняем всю цепочку: препроцессинг + отбор фич + модель
joblib.dump(final_fast_pipeline, 'catboost_pipeline_with_sfs.pkl')

['catboost_pipeline_with_sfs.pkl']

#### Получим список самых важных для обучения фич с исходными именами

In [50]:
# Определим ТОП самых важных, влияющих фич на результат

# def get_feature_importance_original_names(final_pipeline, numeric_columns, non_numeric_columns):
#     # Получаем названия категориальных признаков после OneHot-кодирования
#     categorical_features = final_pipeline.named_steps['preprocessor']\
#                             .named_transformers_['cat']\
#                             .named_steps['onehot']\
#                             .get_feature_names_out(non_numeric_columns)
# 
#     # Все признаки после препроцессинга
#     all_features = np.concatenate([numeric_columns, categorical_features])
# 
#     # Важности признаков от CatBoost
#     importances = final_pipeline.named_steps['model'].get_feature_importance()
# 
#     # DataFrame с признаками и их важностью
#     feature_importance_df = pd.DataFrame({
#         'feature': all_features,
#         'importance': importances
#     })
# 
#     # Для категориальных признаков вернем исходные названия без значений после "_"
#     feature_importance_df['original_feature'] = feature_importance_df['feature'].apply(
#         lambda x: x.split('_')[0] if x in categorical_features else x
#     )
# 
#     # Сгруппируем по оригинальному названию признака и суммируем важность
#     aggregated_importance = feature_importance_df.groupby('original_feature')['importance']\
#                                                  .sum()\
#                                                  .reset_index()\
#                                                  .sort_values(by='importance', ascending=False)\
#                                                  .reset_index(drop=True)
#     return aggregated_importance
# 
# aggregated_importance = get_feature_importance_original_names(final_pipeline, numeric_columns, non_numeric_columns)
# 
# # aggregated_importance.head(50)
# print(list(aggregated_importance['original_feature']))

## 4. Формируем Submission файл для Kaggle обучив модель на 100% данных

In [22]:
final_pipeline.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   IterativeImputer(estimator=RandomForestRegressor(n_estimators=50,
                                                                                                                    random_state=42),
                                                                                    random_state=42))]),
                                                  Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF',...
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir',
       'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC',
       'Fence', 'SaleType', 'SaleCondition'],
      dtype='object'))])),
                ('model',
                 <catboost.core.CatBoostRegressor object at 0x30a232f30>)])

In [23]:
X_submission = test_data.drop(columns=bad_columns)
X_submission.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,Utilities,Neighborhood,Condition1,Condition2,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,NaN,AllPub,NAmes,Feedr,Norm,...,0,0,120,0,NaN,MnPrv,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,NaN,AllPub,NAmes,Norm,Norm,...,0,0,0,0,NaN,NaN,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,NaN,AllPub,Gilbert,Norm,Norm,...,0,0,0,0,NaN,MnPrv,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,NaN,AllPub,Gilbert,Norm,Norm,...,0,0,0,0,NaN,NaN,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,NaN,AllPub,StoneBr,Norm,Norm,...,0,0,144,0,NaN,NaN,1,2010,WD,Normal


In [24]:
y_submission = final_pipeline.predict(X_submission)
y_submission

array([127983.98314642, 165461.52576759, 188011.22594222, ...,
       172546.12618709, 122992.15335977, 225245.79598783])

In [25]:
submission = pd.DataFrame({
    'Id': test_data['Id'],
    'SalePrice': y_submission
})
submission.set_index('Id', inplace=True)
submission.head()

,SalePrice
Id,
1461,127983.983146
1462,165461.525768
1463,188011.225942
1464,193578.988593
1465,187054.207050


In [26]:
submission.to_csv('submission_feature_engineered.csv')

# Выводы
Показатели улучшились! При внедрении IterativeImputera, который использует RandomForestRegression для обучения модели для заполнения пропущенных числовых фич получили прирост к точности.